In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import os
import sys
import math
import warnings
from geopy.geocoders import Nominatim
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM = remove_spaces_in_columns(JLM)

In [5]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['אגף', 'פיקוח', 'סוג_חינוך', 'מעמד_משפטי', 'מספר_כיתות', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM.head()

,סמל_עירייה,סמל_מוסד,שם_מוסד,שלב_חינוך,מכיתה,עד_כיתה,"סה""כ_תלמידים",coordinate_x,coordinate_y
0,21321.00,110106.00,"מ""מ אגרון",יסודי,א,ו,230.00,216876.00,629987.00
1,21323.00,110312.00,"מ""מ ארגנטינה הניסויי",יסודי,א,ח,347.00,216154.00,630603.00
2,21326.00,110155.00,"מ""מ בית הכרם -הישראלי",יסודי,א,ו,534.00,218166.00,632109.00
3,21327.00,110171.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",יסודי,א,ו,331.00,220805.00,629266.00
4,21328.00,111112.00,"מ""מ זלמן ארן",יסודי,א,ו,367.00,220633.00,628408.00


In [6]:
# פונקציה להמרת כיתות לאותיות למספרים
def get_grade_number(grade):
    grade_map = {
        'א': 1, 'ב': 2, 'ג': 3, 'ד': 4, 'ה': 5, 'ו': 6,
        'ז': 7, 'ח': 8, 'ט': 9, 'י': 10, 'יא': 11, 'יב': 12, 
        'יג': 13, 'יד': 14
    }
    return grade_map.get(grade, np.nan)

In [7]:
# חישוב מספר הכיתות
JLM['מספר_כיתות'] = JLM.apply(lambda row: get_grade_number(row['עד_כיתה']) - get_grade_number(row['מכיתה']) + 1, axis=1)
JLM['מספר_כיתות']

# חלוקת התלמידים לפי כיתות
JLM['תלמידים_לכיתה'] = JLM['סה"כ_תלמידים'] / JLM['מספר_כיתות']


In [8]:
# יצירת עמודות עבור כל כיתה
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']
for grade in grades:
    JLM[grade] = JLM.apply(lambda row: row['תלמידים_לכיתה'] if get_grade_number(row['מכיתה']) <= get_grade_number(grade) <= get_grade_number(row['עד_כיתה']) else 0, axis=1)